# This Notebook is used for initial data exploration and filtering for the sales machine learning project.

Most of the EDA has been done using simple print statements, hence not being saved to the script. 

In [ ]:
# Function to make sure the script autostops, in case of wrong lakehouse destination

def lakehouse_gatekeep(expected_db = "ml_curate_lakehouse"):
    db = spark.catalog.currentDatabase()

    if db != expected_db:
       raise Exception(
        f"Forkert Lakehouse er tilkoblet som default: '{db}'."
        f"Det rigtige lakehouse er: '{expected_db}'. Sæt default lakehouse til '{expected_db}' i Explorer i venstre side."
        )
    else:
        print(f"Det korrekte lakehouse: '{db}' er valgt. scriptet fortsættes")

lakehouse_gatekeep()

In [ ]:
#### dependencies ####
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "LEGACY")

In [ ]:
#### Installs if required ####

!pip install scipy

In [ ]:
#### Imports ####
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [ ]:
cols_to_drop = [
    "TrackBatchID",
    "AnalysisKey",
    "BusinessUnitKey",
    "DW_Batch_Created",
    "DealSizeKey",
    "DW_ID",
    "EmployeeKey",
    "LeadCreateDate",
    "LeadCreateDateTime",
    "LeadKey",
    "MovementTypeCRMKey",
    # "OpportunityKey",  # KEEP
    "OpportunityStatusKey",
    "PipelineDate",
    "PrimoDate",
    "ReasonKey",
    "RevenueTypeKey",
    "SalesMotionKey",
    "SalesStageKey",
    "SourceKey",
    "TechnologyKey",
    "WhiteSpot",
    "WinChanceKey",
    "PartitionBy",
    "CRMActionPlanDetailsStatus",
    "CRMActionPlanDetailsKey",
    "CRMCustomerKey",
    "CRMIndustryKey",
    "CRMVendorKey",
    "CurrencyKey",
    "CustomerVoiceAlertKey",
    "CampaignMembersKey",
    "ConsciaCountryKey",
    "CRMBusinessAreaKey",
    "NPSKey",
    "OpportunityDatesKey",
    "OpportunityGroupKey",
    "OpportunityLineKey",
    "OpportunityStateKey",
    "PipelineBreakdownKey",
    "NoOfLeads",
    "NoOfNPSRespons",
    "NoOfOpportunities",
    "NoOfOpportunityLines",
    "NoTransactions",
    "NPSRating",
    "NPSRatingDetractor",
    "NPSRatingPassive",
    "NPSRatingPromoter",
    "NPSStatus",
    "NPSType",
    "CRMActionPlanDetailsCounter",
    "OpportunityRevenueDKK",
    "LeadTechnology",
    "CurrencyID",
    "DevelopDate",
    "OfferDate",
    "NegotiateDate",
    # "CloseDate",  # KEEP
    "DaysBetweenQualifyDevelop",
    "DaysBetweenDevelopOffer",
    "DaysBetweenOfferNegotiate",
    "DaysBetweenNegotiateClose",
    # "Period",  # KEEP
    "WeigthedOpportunityRevenueDKK",
    "PipelineBreakdownValueDKK",
    "LeadsTotalContractValueDKK",
    "Date",
    "SubType2",
    "CRMBusinessLine",
    "CRMServiceOffering",
    "CRMBusinessService",
    # "OpportunityStatusActual",  # KEEP
    # "SalesStageName",  # KEEP
    # "LeadQuality",  # KEEP
    # "DaysInStage",  # KEEP
    # "EstimatedInvoiceDate",  # KEEP
    "SecurityCountry",
    # "WinChanceNumber",  # KEEP
    "Fact_DW_ID"
]

sales_raw_spark = (
    spark.read.table("Sales_ml_raw")
    .filter("CalendarDate IS NULL OR CalendarDate >= '2010-01-01'")
    .drop(*cols_to_drop)
)

sales_raw = sales_raw_spark.toPandas()

In [ ]:
sales_raw.shape

In [ ]:
#### Feature Engineering ####

    # Creating relevant new features from existing irrelevant features. 

    # - Monthly Column instead of CalendarDate
sales_raw['MonthName'] = pd.to_datetime(sales_raw['CalendarDate'], format='%m/%d/%Y, %I:%M:%S %p', errors='coerce').dt.strftime('%B')


In [ ]:
## Importing the budget manually with Minhs excel extract
    # - Need to find a smarter way if its intended to be used in prod due to dynamically changing targets
        # -- Perhaps a dynamically changing xcel spreadsheet where the formats the same

Budget = pd.DataFrame({

    "År": [2024, 2024, 2024, 2025, 2025, 2025, 2025, 2025, 2025, 2025, 2025, 2025],
    "Måned": ['October', 'November', 'December', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September'], 
    "Actual_sol": [329375.14, 323324.67, 485635.66, 309851.72, 300083.53, 336556.5, 302158.6, 305358.2, 443289.2, 447492.9, 9624.7, 0.0],
    "Target_sol": [328937.7, 333129.8, 343623.8, 402712.9, 387175.9, 351229.9, 332012.9, 336120.7, 393550.4, 345422.9, 311976.1, 347876.4],
    "Actual_serv": [42041, 40099, 163547.9, 180604.5, 83752.6, 85229, 40848.2, 46575.4, 66572.8, 196346.8, 744.3, 0.0],
    "Target_serv": [49849.7, 56836.4, 117174.2, 148085.4, 83974.6, 80901.6, 77190.2, 70956.8, 73592.8, 113783.5, 65380.5, 101896.1]
})


In [ ]:
#### Data cleaning ####

    # Removing duplicates, NA-values, irrelevant columns and creating filters

sales_raw = sales_raw.drop_duplicates()

        # Removing every row with negative OpportunityID (seems to be non-specific opps.)

sales_raw = sales_raw[sales_raw['OpportunityKey'] >= 0]
#sales_raw = sales_raw[sales_raw['OpportunityStatusGroup'] == "Closed"]


# Generalizing sales stage down to closed / open

sales_raw['OpportunityStatusGroup'] = sales_raw['OpportunityStatusGroup'].apply(lambda x: 'Closed' if 'Closed' in x else 'Open')


# Config NAs

sales_raw[['NxoLeadStatus', 'LeadQuality', 'LeadStatus', 'LeadStage']] = sales_raw[['NxoLeadStatus', 'LeadQuality', 'LeadStatus', 'LeadStage']].replace("NA", "Not a lead")

sales_raw[['OpportunityStatusActual', 'WinChanceGroup']] = sales_raw[['OpportunityStatusActual', 'WinChanceGroup']].replace("NA", "Its a lead")

sales_raw['TechnologyName'] = sales_raw['TechnologyName'].replace("NA", "Other")

sales_raw[['Title', 'Site', 'Department', 'EmployeeName', 'OpportunityConsciaCountry', 'OpportunitySalesTeam']] = sales_raw[['Title', 'Site', 'Department', 'EmployeeName', 'OpportunityConsciaCountry', 'OpportunitySalesTeam']].replace("NA", "Not stated")

sales_raw['YearName'] = sales_raw['PeriodName'].str.extract(r'(\d{4})').astype(int)

sales_raw = sales_raw[sales_raw['DealSizeName'].notna() & (sales_raw['DealSizeName'] != "NA")] 

## Filteirng out loads of NA columns (probably fom misclicks in CRM, committing faulty data entries)

sales_raw = sales_raw[sales_raw['RevenueType'] != 'NA']

# Converting to absolute values


sales_raw['DaysToClose'] = sales_raw['DaysToClose'].abs()

# Columndropping

sales_clean = sales_raw.drop(columns = {
    'OpportunityKey',
    'OpportunityStatusActual',
    'LeadQuality',
    'CloseDate',
    'Period',
    'SalesStageName',
    'DaysInStage',
    'EstimatedInvoiceDate',
    'WinChanceNumber'
})

    # Dropping different currency values (Keeping it separate incase i want to swap currency later)
sales_clean = sales_clean.drop(columns = {
    #ACV
    'AnnualContractValueEUR',
    'AnnualContractValueGBP',
    'AnnualContractValueLocal',
    'AnnualContractValueNOK',
    'AnnualContractValueSEK',
    'AnnualContractValueUSD',
    #LeadTCV
    'LeadsTotalContractValueEUR',
    'LeadsTotalContractValueGBP',
    'LeadsTotalContractValueLocal',
    'LeadsTotalContractValueNOK',
    'LeadsTotalContractValueSEK',
    'LeadsTotalContractValueUSD',
    #Margins
    'MarginValueEUR',
    'MarginValueGBP',
    'MarginValueLocal',
    'MarginValueNOK',
    'MarginValueSEK',
    'MarginValueUSD',
    #PipelineBreakdownValue,
    'PipelineBreakdownValueEUR',
    'PipelineBreakdownValueGBP',
    'PipelineBreakdownValueLocal',
    'PipelineBreakdownValueNOK',
    'PipelineBreakdownValueSEK',
    'PipelineBreakdownValueUSD',
    #TCV,
    'TotalContractValueEUR',
    'TotalContractValueGBP',
    'TotalContractValueLocal',
    'TotalContractValueNOK',
    'TotalContractValueSEK',
    'TotalContractValueUSD',
    #WeightedACV
    'WeightedAnnualContractValueEUR',
    'WeightedAnnualContractValueGBP',
    'WeightedAnnualContractValueLocal',
    'WeightedAnnualContractValueNOK',
    'WeightedAnnualContractValueSEK',
    'WeightedAnnualContractValueUSD',
    #WeightedMargin
    'WeigthedMarginValueEUR',
    'WeigthedMarginValueGBP',
    'WeigthedMarginValueLocal',
    'WeigthedMarginValueNOK',
    'WeigthedMarginValueSEK',
    'WeigthedMarginValueUSD',
    #WeightedOpportunityRevenue
    'WeigthedOpportunityRevenueEUR',
    'WeigthedOpportunityRevenueGBP',
    'WeigthedOpportunityRevenueLocal',
    'WeigthedOpportunityRevenueNOK',
    'WeigthedOpportunityRevenueSEK',
    'WeigthedOpportunityRevenueUSD',
    #WeightedTCV
    'WeigthedTotalContractValueEUR',
    'WeigthedTotalContractValueGBP',
    'WeigthedTotalContractValueLocal',
    'WeigthedTotalContractValueNOK',
    'WeigthedTotalContractValueSEK',
    'WeigthedTotalContractValueUSD',
    #OpportunityRevenue
    'OpportunityRevenueEUR',
    'OpportunityRevenueGBP',
    'OpportunityRevenueLocal',
    'OpportunityRevenueNOK',
    'OpportunityRevenueSEK',
    'OpportunityRevenueUSD',
    # Might need to get readded
    #'WeigthedMarginValueDKK',
    #'WeightedAnnualContractValueDKK',
    #'WeigthedTotalContractValueDKK'

})



In [ ]:
sales_clean['OpportunityStatusGroup'].value_counts()

In [ ]:
# Reordering columns for overview
sales_clean = sales_clean[
    [
       'OpportunityStatusGroup', 'CRMBusinessArea', 'OpportunityID', 'OpportunityNumber','OpportunityName', 'OpportunityState','TotalContractValueDKK', 'AnnualContractValueDKK',
        'MarginValueDKK',

        'CalendarDate', 'PeriodName', 'MonthName', 'YearName', 'CreatedDate', 'QualifyDate', 'ActualClosedDate',

        "EmployeeName", "Title", "Department", "Site",

        'CurrentSalesStage', 'OpportunityConsciaCountry',
        'OpportunitySalesTeam', 'DaysBetweenCreateClose', 'RevenueType', 'SubType',

        'LeadStage', 'LeadStatus', 'NxoLeadStatus',

        'DaysToClose', 'DealSizeName', 'IndustryName',
        'OpportunityStatusActualGroup',
        'SecurityCountryName', 'TechnologyName', 'WinChanceName', 'WinChanceGroup', 'EstimatedCloseDate'
    ]
]




In [ ]:
sales_lead = sales_clean[sales_clean['LeadStatus'] != 'Not a lead']
sales_opp = sales_clean[sales_clean['LeadStatus'] == 'Not a lead']

In [ ]:
# Flattening Feature engineering to not lose lead and stage information about opportunities: 


         # does the opp. stem from a lead?
sales_opp['FromLead'] = sales_opp['OpportunityNumber'].isin(sales_lead['OpportunityNumber']).map({True: 'Yes', False: 'No'})



        # Days Between Marketing Qualified Lead - Sales Qualified Lead (essentially capturing the span of the lead duration)

        # Doesnt work as its somehow dimensionally enhanced?.., and leads dont seem to be important (Minhs words) so removed for now
#lead_span = ( sales_lead.groupby('OpportunityNumber')['CalendarDate'].agg(['min', 'max']) .reset_index())
#lead_span['DaysBetweenMQL_SQL'] = (lead_span['max'] - lead_span['min']).dt.days

#sales_opp = sales_opp.merge(lead_span[['OpportunityNumber', 'DaysBetweenMQL_SQL']], on = 'OpportunityNumber', how = 'left')

#sales_opp['DaysBetweenMQL_SQL'] = sales_opp['DaysBetweenMQL_SQL'].fillna(-1)


# turning every monetary column into absolute values 

mon_cols = [
    'TotalContractValueDKK',
   # 'WeigthedMarginValueDKK',
    'AnnualContractValueDKK',
   #'WeightedAnnualContractValueDKK',
    #'WeigthedTotalContractValueDKK',
    'MarginValueDKK'
]

sales_opp[mon_cols] = sales_opp[mon_cols].abs()

## A lot of config for WinChance cols

sales_opp['PeriodDate'] = pd.to_datetime(sales_opp['PeriodName'], format='%B %Y')
sales_opp = sales_opp.sort_values(by=['OpportunityNumber', 'PeriodDate'])

sales_opp['WinChanceNum'] = (
    sales_opp['WinChanceName']
    .astype(str)
    .str.extract(r'(\d+)\s?%')[0]
    .astype(float)
    .fillna(0)
)

Actual_WinChance = (
    sales_opp[sales_opp['OpportunityState'] == 'Won']
    .sort_values(by=['OpportunityNumber', 'PeriodDate'])
    .groupby('OpportunityNumber')['WinChanceNum']
    .first()
    .reset_index()
    .rename(columns={'WinChanceNum': 'ActualWinChance'})
)





In [ ]:
sales_flattened = sales_opp[sales_opp['OpportunityStatusGroup'] == 'Closed']

# Grundet diverse data entries pr. cloesd row (tror det CRM dupe fejl) er det nødsaget at filtrere mere manuelt gennem PeriodName

sales_flattened['PeriodDate'] = pd.to_datetime(sales_flattened['PeriodName'], format = '%B %Y')
sales_flattened_sorted = sales_flattened.sort_values(by = ['OpportunityNumber', 'PeriodDate'], ascending = [True, False])
sales_flattened = sales_flattened_sorted.drop_duplicates(subset = 'OpportunityNumber', keep = 'first')

## Flattening out the WinChance columns to eliminate dataleakage
        # In addition, the winchance suffers from extreme amounts of label leakage, as the latest dataentry for a opportunity is typically set to 100% winchance once the opportunity is won, although thats not the winchance that was current, when the opportunity was won. 
                # As I dont want the model to learn from artificially inflated datapoints, I'll rearrange the winchance data to include summary-like variables and the correct winchance at the moment the opportunity got marked as won. 
                    # Once again, in addition to that, The winChance is labelproxy, essentially describibing what im trying to predict. This can cause the model to be very reliant on this specific feature, therefore splitting it into a summary is generally very good. 

def winchance_summarised(group):
    group = group.sort_values('PeriodDate') 
    WinChance = group['WinChanceNum'].astype(float).values
    dates = group['PeriodDate']
    days = (dates - dates.iloc[0]).dt.days

    max_win = WinChance.max()

    won_rows = group[group['OpportunityState'] == 'Won']
    current_win = won_rows['WinChanceNum'].iloc[0] if not won_rows.empty else WinChance[-1]

    return pd.Series({
        'MinWinChance': WinChance.min(),
        'MaxWinChance': max_win,
        'stdWinChance': WinChance.std(),
        'CurrentWinChance': current_win,
        'IsCurrentWinChanceMax': current_win == max_win,
        'WinChanceDropFromMax': max_win - current_win
    })

summary_opp = sales_opp.groupby('OpportunityNumber').apply(winchance_summarised).reset_index()

sales_flattened = sales_flattened.merge(summary_opp, on = 'OpportunityNumber', how = "left")

# sales_flattened = sales_flattened.drop(columns = 'PeriodDate')

sales_flattened = sales_flattened.merge(Actual_WinChance, on='OpportunityNumber', how='left')
sales_flattened['ActualWinChance'] = sales_flattened.apply(
    lambda row: row['ActualWinChance'] if row['OpportunityState'] == 'Won' else row['WinChanceNum'],
    axis=1
)


# Fjerner de resterende blanke og missing opps.

sales_flattened = sales_flattened[
    sales_flattened['OpportunityNumber'].notna() &
    (sales_flattened['OpportunityNumber'].astype(str).str.strip().str.upper() != 'NA') &
    (sales_flattened['OpportunityNumber'].astype(str).str.strip() != '')
]

In [ ]:
# Removing potential dataleakage and the last unnessecary cols

sales_flattened = sales_flattened.drop(columns = {
    'DaysToClose',
    #'CreatedDate',
    'QualifyDate',
    #'ActualClosedDate',
    'CurrentSalesStage',
    'RevenueType',
    'OpportunityStatusActualGroup',
        # No opps with leads after filtering
    'LeadStatus',
    'NxoLeadStatus',
    'FromLead',
    'LeadStage',
    'CurrentWinChance',
    'WinChanceNum',
    'WinChanceName',
    'WinChanceGroup',
    
})

sales_flattened.rename(columns = {
    'ActualWinChance': 'WinChance'
})



In [ ]:
solutions_raw = sales_flattened[sales_flattened['CRMBusinessArea'] == "Solutions"]
service_raw = sales_flattened[sales_flattened['CRMBusinessArea'] == "Services"]

solutions_ml = solutions_raw.drop(columns = {
    'CRMBusinessArea',
    'OpportunityID',
    'OpportunityName',
    #'CalendarDate',
    'MaxWinChance',
    'ActualWinChance',
    'WinChanceDropFromMax',
    'MinWinChance',
    'stdWinChance',
    'IsCurrentWinChanceMax',
    'PeriodName',
    'SecurityCountryName'

})



# Define the columns and their intended data types
Dtypes_to_change = {
    'TotalContractValueDKK': 'float',       
    'AnnualContractValueDKK': 'float',
    'MarginValueDKK': 'float',
    #'PeriodName': 'category',
    'MonthName': 'category',
    'EmployeeName': 'category',
    'Title': 'category',
    'Department': 'category',
    'Site': 'category',
    'OpportunityStatusGroup': 'category',
    'OpportunityConsciaCountry': 'category',
    'OpportunitySalesTeam': 'category',
    'SubType': 'category',
    'DealSizeName': 'category',
    'IndustryName': 'category',
    'TechnologyName': 'category',
    'YearName': 'category'



}

# Apply the dtype conversions
for column, dtype in Dtypes_to_change.items():
    try:
        solutions_ml[column] = solutions_ml[column].astype(dtype)
    except Exception as e:
        print(f"Could not convert {column} to {dtype}: {e}")


solutions_ml['DaysBetweenCreateClose'] = solutions_ml['DaysBetweenCreateClose'].abs()

otherCRM_opps = solutions_ml.loc[
    solutions_ml['DaysBetweenCreateClose'] == 0,
    'OpportunityNumber'
].tolist()

solutions_ml = solutions_ml[solutions_ml['DaysBetweenCreateClose'] != 0]

solutions_ml = solutions_ml[solutions_ml['TotalContractValueDKK'] != 0]


In [ ]:
solutions_ml.shape

In [ ]:
##################################################
#### Script copy but for active opportunities ####
##################################################

sales_flattened_active = sales_opp[sales_opp['OpportunityStatusGroup'] == 'Open']


sales_flattened_active['PeriodDate'] = pd.to_datetime(sales_flattened_active['PeriodName'], format = '%B %Y')
sales_flattened_active_sorted = sales_flattened_active.sort_values(by = ['OpportunityNumber', 'PeriodDate'], ascending = [True, False])
sales_flattened_active = sales_flattened_active_sorted.drop_duplicates(subset = 'OpportunityNumber', keep = 'first')

## Flattening out the WinChance columns to eliminate dataleakage
        # In addition, the winchance suffers from extreme amounts of label leakage, as the latest dataentry for a opportunity is typically set to 100% winchance once the opportunity is won, although thats not the winchance that was current, when the opportunity was won. 
                # As I dont want the model to learn from artificially inflated datapoints, I'll rearrange the winchance data to include summary-like variables and the correct winchance at the moment the opportunity got marked as won. 
                    # Once again, in addition to that, The winChance is labelproxy, essentially describibing what im trying to predict. This can cause the model to be very reliant on this specific feature, therefore splitting it into a summary is generally very good. 



sales_flattened_active = sales_flattened_active.merge(summary_opp, on = 'OpportunityNumber', how = "left")

# sales_flattened_active = sales_flattened_active.drop(columns = 'PeriodDate')

sales_flattened_active = sales_flattened_active.merge(Actual_WinChance, on='OpportunityNumber', how='left')
sales_flattened_active['ActualWinChance'] = sales_flattened_active.apply(
    lambda row: row['ActualWinChance'] if row['OpportunityState'] == 'Won' else row['WinChanceNum'],
    axis=1
)

# Fjerner de resterende blanke og missing opps.

sales_flattened_active = sales_flattened_active[
    sales_flattened_active['OpportunityNumber'].notna() &
    (sales_flattened_active['OpportunityNumber'].astype(str).str.strip().str.upper() != 'NA') &
    (sales_flattened_active['OpportunityNumber'].astype(str).str.strip() != '')
]


# Removing potential dataleakage and the last unnessecary cols

sales_flattened_active = sales_flattened_active.drop(columns = {
    'DaysToClose',
    #'CreatedDate',
    'QualifyDate',
    #'ActualClosedDate',
    'CurrentSalesStage',
    'RevenueType',
    'OpportunityStatusActualGroup',
        # No opps with leads after filtering
    'LeadStatus',
    'NxoLeadStatus',
    'FromLead',
    'LeadStage',
    'CurrentWinChance',
    'WinChanceNum',
    'WinChanceName',
    'WinChanceGroup'
})

sales_flattened_active.rename(columns = {
    'ActualWinChance': 'WinChance'
})

solutions_raw_active = sales_flattened_active[sales_flattened_active['CRMBusinessArea'] == "Solutions"]
service_raw_active = sales_flattened_active[sales_flattened_active['CRMBusinessArea'] == "Services"]

solutions_ml_active = solutions_raw_active.drop(columns = {
    'CRMBusinessArea',
    'OpportunityID',
   # 'OpportunityNumber',
    'OpportunityName',
    #'OpportunityStatusHistoryGroup',
    #'OpportunityStatus', # forgot to remove in EDA script, can be fixed
    #'CalendarDate',
    'MaxWinChance',
    'ActualWinChance',
    'WinChanceDropFromMax',
    'MinWinChance',
    'stdWinChance',
    'IsCurrentWinChanceMax',
    'PeriodName',
    'SecurityCountryName'

})



# Define the columns and their intended data types
Dtypes_to_change = {
    'TotalContractValueDKK': 'float',       
    'AnnualContractValueDKK': 'float',
    'MarginValueDKK': 'float',
    #'PeriodName': 'category',
    'MonthName': 'category',
    'EmployeeName': 'category',
    'Title': 'category',
    'Department': 'category',
    'Site': 'category',
    'OpportunityStatusGroup': 'category',
    'OpportunityConsciaCountry': 'category',
    'OpportunitySalesTeam': 'category',
    'SubType': 'category',
    'DealSizeName': 'category',
    'IndustryName': 'category',
    'TechnologyName': 'category',
    'YearName': 'category'



}

# Apply the dtype conversions
for column, dtype in Dtypes_to_change.items():
    try:
        solutions_ml_active[column] = solutions_ml_active[column].astype(dtype)
    except Exception as e:
        print(f"Could not convert {column} to {dtype}: {e}")


solutions_ml_active['DaysBetweenCreateClose'] = solutions_ml_active['DaysBetweenCreateClose'].abs()

solutions_ml_active = solutions_ml_active[solutions_ml_active['DaysBetweenCreateClose'] != 0]

solutions_ml_active['EstimatedCloseDate'] = pd.to_datetime(solutions_ml_active['EstimatedCloseDate'].astype(str), format = '%Y%m%d')




In [ ]:
# Removing IC
solutions_ml = solutions_ml[~solutions_ml['IndustryName']
                            .str.strip()
                            .str.upper()
                            .str.startswith('IC')]

solutions_ml_active = solutions_ml_active[~solutions_ml_active['IndustryName']
                            .str.strip()
                            .str.upper()
                            .str.startswith('IC')]

In [ ]:
## Descriptive statistics for numerical values prior to outlier-handling

solutions_ml.describe().T

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Dine valgte numeriske features
numvars = [
    'TotalContractValueDKK',
    'AnnualContractValueDKK',
    'MarginValueDKK',
    'DaysBetweenCreateClose',
]

# Opret subplots: 4 rækker, 1 kolonne
fig, axes = plt.subplots(nrows=len(numvars), ncols=1, figsize=(12, 4*len(numvars)))

for i, col in enumerate(numvars):
    ax = axes[i]
    sns.histplot(solutions_ml[col], bins=30, kde=False, ax=ax)
    
    # Formater x-aksen til “normale tal” med tusind-separator
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))
    
    ax.set_title(f'Distribution af {col} forud for outlier håndtering')
    ax.set_xlabel(col)
    ax.set_ylabel('Antal')

plt.tight_layout()
plt.show()



In [ ]:
solutions_outlier_free = solutions_ml.copy()

for var in numvars:
    high_q = solutions_ml[var].quantile(0.98)
    low_q = solutions_ml[var].quantile(0.02)

    if var == "DaysBetweenCreateClose":
        # Kun øvre cutoff for denne variabel
        solutions_outlier_free = solutions_outlier_free[
            solutions_outlier_free[var] <= high_q
        ]
    else:
        # Normal 2%-98% cutoff
        solutions_outlier_free = solutions_outlier_free[
            (solutions_outlier_free[var] >= low_q) &
            (solutions_outlier_free[var] <= high_q)
        ]

summary_før = solutions_ml[numvars].describe().T
summary_efter = solutions_outlier_free[numvars].describe().T

print("Før fjernet outliers:\n", summary_før)
print("\nEfter outliers:\n", summary_efter)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Dine valgte numeriske features
numvars = [
    'TotalContractValueDKK',
    'AnnualContractValueDKK',
    'MarginValueDKK',
    'DaysBetweenCreateClose'
]

# Opret subplots: 4 rækker, 1 kolonne
fig, axes = plt.subplots(nrows=len(numvars), ncols=1, figsize=(12, 4*len(numvars)))

for i, col in enumerate(numvars):
    ax = axes[i]
    sns.histplot(solutions_outlier_free[col], bins=30, kde=False, ax=ax)
    
    # Formater x-aksen til “normale tal” med tusind-separator
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))
    
    ax.set_title(f'Histogram of {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()


In [ ]:
for var in numvars:
    high_q = solutions_ml[var].quantile(0.98)
    low_q = solutions_ml[var].quantile(0.02)

    if var == "DaysBetweenCreateClose":
        # Kun øvre outliers fjernes
        solutions_ml = solutions_ml[solutions_ml[var] <= high_q]
    else:
        # Fjern både nedre og øvre outliers
        solutions_ml = solutions_ml[
            (solutions_ml[var] >= low_q) & 
            (solutions_ml[var] <= high_q)
        ]


In [ ]:
#### Tester kategoriske variable
import matplotlib.pyplot as plt
import seaborn as sns

cat_cols = solutions_ml.select_dtypes(include=['object', 'category']).columns.tolist()

fig, axes = plt.subplots(nrows=len(cat_cols), ncols=1, figsize=(14, 5 * len(cat_cols)))

if len(cat_cols) == 1:
    axes = [axes]

for i, col in enumerate(cat_cols):
    ax = axes[i]
    
    # Freq table
    freq = solutions_ml[col].value_counts(dropna=False)
    perc = solutions_ml[col].value_counts(normalize=True, dropna=False) * 100
    
    # combination
    freq_df = (
        pd.DataFrame({'count': freq, 'percent': perc})
        .sort_values('percent', ascending=False)
    )
    
    # Barplot
    sns.barplot(
        x=freq_df.index.astype(str),
        y=freq_df['percent'],
        ax=ax
    )
    
    ax.set_title(f"Kategorisk fordeling for {col}")
    ax.set_ylabel("Procent (%)")
    ax.set_xlabel(col)
    ax.tick_params(axis='x', rotation=45)
    
    # % label
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.1f}%'))

plt.tight_layout()
plt.show()


In [ ]:
# Vi opelever enkelte kategorier med en enstydelig meget dominerede variabel, derfor fjerner vi:
    # - Title
    # - Department
    # - Site
    # - Der kan argumenteres for OpportunitySalesteam, med en enstydning repræsenteret kategori på ~ 60%


solutions_ml = solutions_ml.drop(columns = [
   
    'Title', 'Department', 'Site', 'IndustryName', 'OpportunitySalesTeam'

])

solutions_ml_active = solutions_ml_active.drop(columns = [
   
    'Title', 'Department', 'Site', 'IndustryName', 'OpportunitySalesTeam'

])

In [ ]:

#### Feature engineering more variables to capture trends not present within the data as of now ####

# mapping for month-column

month_map = {
    "January": 1, "February": 2, "March": 3,
    "April": 4, "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}

# For closed opportunities
solutions_ml['TCV_Margin_Ratio'] = solutions_ml['MarginValueDKK'] / solutions_ml['TotalContractValueDKK']
solutions_ml['TCV_Margin_Ratio'].replace([np.inf, -np.inf], 0, inplace=True)
solutions_ml['MonthNum'] = solutions_ml['MonthName'].map(month_map)
solutions_ml['MonthNum'] = solutions_ml['MonthNum'].astype(int)
solutions_ml['Quarter'] = ((solutions_ml['MonthNum'] -1) // 3 + 1)
solutions_ml['Is_EOQ'] = solutions_ml['MonthNum'].isin([3,6,9,12]).astype(int)

# For open opportunities
solutions_ml_active['TCV_Margin_Ratio'] = solutions_ml_active['MarginValueDKK'] / solutions_ml_active['TotalContractValueDKK']
solutions_ml_active['MonthNum'] = solutions_ml_active['MonthName'].map(month_map)
solutions_ml_active['MonthNum'] = solutions_ml_active['MonthNum'].astype(int)
solutions_ml_active['Quarter'] = ((solutions_ml_active['MonthNum'] -1) // 3 + 1)
solutions_ml_active['Is_EOQ'] = solutions_ml_active['MonthNum'].isin([3,6,9,12]).astype(int)

# Cumulative Employee features
    # As the data needs to be cumulative, i need to concat the closed + open dataset together again
        # this is a lazy fix. The better fix is to redo some of the previous filtering to allow for introductoin of feature engineering earlier on in the script

samlet_sol = pd.concat([solutions_ml, solutions_ml_active], ignore_index = True)
samlet_sol['PeriodMonth'] = pd.to_datetime(samlet_sol['PeriodDate'], format = "%B %Y").dt.to_period("M")
samlet_sol = samlet_sol.sort_values(['EmployeeName', 'PeriodMonth'])
samlet_sol['IsClosed'] = (samlet_sol['OpportunityStatusGroup'] == "Closed").astype(int)
samlet_sol['IsWon'] = (samlet_sol['OpportunityState'] == "Won").astype(int)

samlet_sol['Total_closed_employee'] = (
    samlet_sol.groupby('EmployeeName')
    .apply(lambda g: g['IsClosed'].cumsum().shift(1).fillna(0))
    .reset_index(level = 0, drop = True)
)

samlet_sol['Total_won_employee'] = (
    samlet_sol.groupby('EmployeeName')
    .apply(lambda g: g['IsWon'].cumsum().shift(1).fillna(0))
    .reset_index(level = 0, drop = True)
)


no_employee_name = samlet_sol['EmployeeName'] == "Not Available"

samlet_sol.loc[no_employee_name, [
    'Total_closed_employee',
    'Total_won_employee'
]] = -1



## Aggregating opportunities x monthly, then employee x monthly; creating monthly lag features

samlet_sol = samlet_sol.sort_values(['PeriodMonth'])

monthly_closed = (
    samlet_sol.groupby('PeriodMonth')['IsClosed']
    .sum()
    .reset_index()
    .rename(columns = {'IsClosed': 'closed_per_month'})
)

monthly_closed['lag_1m_closed'] = monthly_closed['closed_per_month'].shift(1).fillna(0)
monthly_closed['lag_3m_closed'] = monthly_closed['closed_per_month'].shift(3).fillna(0)
monthly_closed['lag_year_closed'] = monthly_closed['closed_per_month'].shift(12).fillna(0)

monthly_closed['sum_3m_closed'] = (
    monthly_closed['closed_per_month']
    .rolling(window=3, min_periods=1)
    .sum()
    .shift(1)
    .fillna(0)
)

monthly_closed['sum_12m_closed'] = (
    monthly_closed['closed_per_month']
    .rolling(window=12, min_periods=1)
    .sum()
    .shift(1)
    .fillna(0)
)


monthly_won = (
    samlet_sol.groupby('PeriodMonth')['IsWon']
    .sum()
    .reset_index()
    .rename(columns={'IsWon': 'won_per_month'})
)

monthly_won['lag_1m_won'] = monthly_won['won_per_month'].shift(1).fillna(0)
monthly_won['lag_3m_won'] = monthly_won['won_per_month'].shift(3).fillna(0)
monthly_won['lag_year_won'] = monthly_won['won_per_month'].shift(12).fillna(0)

monthly_won['sum_3m_won'] = (
    monthly_won['won_per_month']
    .rolling(window=3, min_periods=1)
    .sum()
    .shift(1)
    .fillna(0)
)

monthly_won['sum_12m_won'] = (
    monthly_won['won_per_month']
    .rolling(window=12, min_periods=1)
    .sum()
    .shift(1)
    .fillna(0)
)



employee_monthly = (
    samlet_sol.groupby(['EmployeeName', 'PeriodMonth'])['IsClosed']
    .sum()
    .reset_index()
    .rename(columns = {'IsClosed': 'closed_monthly_employee'})
)

employee_monthly['emp_lag_1m_closed'] = employee_monthly.groupby('EmployeeName')['closed_monthly_employee'].shift(1).fillna(0)
employee_monthly['emp_lag_3m_closed'] = employee_monthly.groupby('EmployeeName')['closed_monthly_employee'].shift(3).fillna(0)
employee_monthly['emp_lag_12m_closed'] = employee_monthly.groupby('EmployeeName')['closed_monthly_employee'].shift(12).fillna(0)

employee_monthly['emp_sum_3m_closed'] = (
    employee_monthly.groupby('EmployeeName')['closed_monthly_employee']
    .rolling(window=3, min_periods=1)
    .sum()
    .shift(1)
    .reset_index(level=0, drop=True)
    .fillna(0)
)

employee_monthly['emp_sum_12m_closed'] = (
    employee_monthly.groupby('EmployeeName')['closed_monthly_employee']
    .rolling(window=12, min_periods=1)
    .sum()
    .shift(1)
    .reset_index(level=0, drop=True)
    .fillna(0)
)


employee_won_monthly = (
    samlet_sol.groupby(['EmployeeName', 'PeriodMonth'])['IsWon']
    .sum()
    .reset_index()
    .rename(columns={'IsWon': 'won_monthly_employee'})
)

employee_won_monthly['emp_lag_1m_won'] = employee_won_monthly.groupby('EmployeeName')['won_monthly_employee'].shift(1).fillna(0)
employee_won_monthly['emp_lag_3m_won'] = employee_won_monthly.groupby('EmployeeName')['won_monthly_employee'].shift(3).fillna(0)
employee_won_monthly['emp_lag_12m_won'] = employee_won_monthly.groupby('EmployeeName')['won_monthly_employee'].shift(12).fillna(0)

employee_won_monthly['emp_sum_3m_won'] = (
    employee_won_monthly.groupby('EmployeeName')['won_monthly_employee']
    .rolling(window=3, min_periods=1)
    .sum()
    .shift(1)
    .reset_index(level=0, drop=True)
    .fillna(0)
)

employee_won_monthly['emp_sum_12m_won'] = (
    employee_won_monthly.groupby('EmployeeName')['won_monthly_employee']
    .rolling(window=12, min_periods=1)
    .sum()
    .shift(1)
    .reset_index(level=0, drop=True)
    .fillna(0)
)

##########################################################
#### Merging lag & roll features back into samlet_sol ####
##########################################################

samlet_sol = samlet_sol.merge(
    monthly_closed[
        ['PeriodMonth',
        'lag_1m_closed', 'lag_3m_closed', 'lag_year_closed',
        'sum_3m_closed', 'sum_12m_closed']],
    on='PeriodMonth',
    how='left'
)

samlet_sol = samlet_sol.merge(
    monthly_won[
        ['PeriodMonth',
        'lag_1m_won', 'lag_3m_won', 'lag_year_won',
        'sum_3m_won', 'sum_12m_won']],
    on='PeriodMonth',
    how='left'
)

samlet_sol = samlet_sol.merge(
    employee_monthly[
        ['EmployeeName', 'PeriodMonth',
        'closed_monthly_employee',
        'emp_lag_1m_closed', 'emp_lag_3m_closed', 'emp_lag_12m_closed',
        'emp_sum_3m_closed', 'emp_sum_12m_closed']],
    on=['EmployeeName', 'PeriodMonth'],
    how='left'
)

samlet_sol = samlet_sol.merge(
    employee_won_monthly[
        ['EmployeeName', 'PeriodMonth',
        'won_monthly_employee',
        'emp_lag_1m_won', 'emp_lag_3m_won', 'emp_lag_12m_won',
        'emp_sum_3m_won', 'emp_sum_12m_won']],
    on=['EmployeeName', 'PeriodMonth'],
    how='left'
)




## Defaulting values to -1 in the instance of no stated EmployeeName, as i dont want the model to learn patterns around a non-existant Employee.

emp_lag_roll_features = [
    'emp_lag_1m_closed', 'emp_lag_3m_closed', 'emp_lag_12m_closed', 
    'emp_sum_3m_closed', 'emp_sum_12m_closed',
    'emp_lag_1m_won', 'emp_lag_3m_won', 'emp_lag_12m_won',
    'emp_sum_3m_won', 'emp_sum_12m_won'
]
samlet_sol.loc[samlet_sol['EmployeeName'] == "Not Available", emp_lag_roll_features] = -1



##############################################################
#### Merging back into solutions_ml & solutions_ml_active ####
##############################################################

lag_roll_features = [
    'lag_1m_closed', 'lag_3m_closed', 'sum_3m_closed', 'sum_12m_closed',
    'lag_1m_won', 'lag_3m_won', 'sum_3m_won', 'sum_12m_won',
    'emp_lag_1m_closed', 'emp_lag_3m_closed', 'emp_lag_12m_closed', 'emp_sum_3m_closed', 'emp_sum_12m_closed',
    'emp_lag_1m_won', 'emp_lag_3m_won', 'emp_lag_12m_won', 'emp_sum_3m_won', 'emp_sum_12m_won'
]

solutions_ml = solutions_ml.merge(
    samlet_sol[['EmployeeName', 'OpportunityNumber'] + lag_roll_features],
    on=['EmployeeName', 'OpportunityNumber'],
    how='left'
)

solutions_ml_active = solutions_ml_active.merge(
    samlet_sol[['EmployeeName', 'OpportunityNumber'] + lag_roll_features],
    on=['EmployeeName', 'OpportunityNumber'],
    how='left'
)




# Drop  columns used for feature engineering and filtering within the script.
    # Likewise dropping the redundant closedate for each closed and open opportunities (Estimated for closed, actualclosedate for open)

solutions_ml = solutions_ml.drop(columns = [
    'MonthNum', 'EstimatedCloseDate', 'PeriodDate', 'CalendarDate', 'CreatedDate'
])

solutions_ml_active = solutions_ml_active.drop(columns = [
   'MonthNum', 'ActualClosedDate', 'PeriodDate', 'CalendarDate', 'CreatedDate'
])


# To ensure same columns for machine learning, we need to rename the columns of ActualClosedDate and EstimatedCloseDate
solutions_ml = solutions_ml.rename(columns = {'ActualClosedDate': 'CloseDate'})
solutions_ml_active = solutions_ml_active.rename(columns = {'EstimatedCloseDate': 'CloseDate'})





In [ ]:
# Corrplot to check correlation between target var + numvars aswell as checking for multicol
# Although decision trees are naturally prone to multicol it never hurts to test and understand the data more nuanced



lag_roll_features = [
    'lag_1m_closed', 'lag_3m_closed', 'sum_3m_closed', 'sum_12m_closed',
    'lag_1m_won', 'lag_3m_won', 'sum_3m_won', 'sum_12m_won',
    'emp_lag_1m_closed', 'emp_lag_3m_closed', 'emp_lag_12m_closed', 'emp_sum_3m_closed', 'emp_sum_12m_closed',
    'emp_lag_1m_won', 'emp_lag_3m_won', 'emp_lag_12m_won', 'emp_sum_3m_won', 'emp_sum_12m_won'
]

corr_solutions = solutions_ml.copy()

corr_solutions['OpportunityState'] = corr_solutions['OpportunityState'].map({
    'Won': 1,
    'Lost': 0
})



corr_df = corr_solutions[lag_roll_features]


# Compute correlation matrix (Pearson by default)
corr_matrix = corr_df.corr()

# Plot

fig,ax = plt.subplots(figsize = (10,10))

mask = np.triu(np.ones_like(corr_matrix, dtype = bool), k = 1)

sns.heatmap(
    corr_matrix,
    mask = mask,
    annot = False,
    fmt = ".2f",
    cmap = "RdBu_r",
    center = 0,
    vmin = -1,
    vmax = 1,
    square = True,
    linewidth = 0.5,
    annot_kws={"size": 8, "color": "black", "weight": "bold"},
    ax = ax,
    clip_on = False
)

plt.title("Korrelationsmatrice over lag / roll features", fontsize = 14, fontweight = "bold")
plt.tight_layout()
plt.show()


In [ ]:
# Heavy ocrrelation between a lot of the newly added lag / roll features; therefore we remove a lot of these features to maintain only a few

features_to_keep = [
    'sum_3m_won',
    'sum_12m_won',
    'emp_lag_1m_won',
    'emp_lag_12m_won',
    'emp_sum_3m_won'
]

# Drop alle de andre lag/roll features
solutions_ml = solutions_ml.drop(columns=[col for col in lag_roll_features if col not in features_to_keep])
solutions_ml_active = solutions_ml_active.drop(columns=[col for col in lag_roll_features if col not in features_to_keep])


# Fjerner samtidigt andre korrelerede features;
    # - AnnualContractValueDKK (korreleret med TCV)
    # - Margin (har lavet TCV_To_Margin_Ratio variabel. Denne burde være redundant nu)

solutions_ml = solutions_ml.drop(columns = [
   # 'AnnualContractValueDKK'#,
    #'MarginValueDKK'
])

solutions_ml_active = solutions_ml_active.drop(columns = [
    #'AnnualContractValueDKK'#,
    #'MarginValueDKK'
])

In [ ]:
## skubber dataframe til Lakehouse

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

#spark.createDataFrame(sales_flattened).write.format("delta").mode("overwrite").saveAsTable("sales_flat")
        # Fix at a later point. not important now

spark.createDataFrame(solutions_ml).write.format("delta").mode("overwrite").saveAsTable("solutions_ml")
spark.createDataFrame(solutions_ml_active).write.format("delta").mode("overwrite").saveAsTable("Active_opportunities")






# Explorative Data analysis

In [ ]:
# As the categorical dtypes default to objects once converted back and forth between spark/pandas dataframes, its needed to once again transform the dtypes

for col in solutions_ml.select_dtypes(include = 'object').columns:
    solutions_ml[col] = solutions_ml[col].astype('category')

solutions_ml['Is_EOQ'] = solutions_ml['Is_EOQ'].astype('category') 
solutions_ml['Quarter'] = solutions_ml['Quarter'].astype('category') 


# Likewise some of the correlation testing and plotting requires the target variable to numerical

solutions_ml['OpportunityState_num'] = solutions_ml['OpportunityState'].map({'Won': 1, 'Lost': 0})


# Other dependencies for EDA

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import chi2_contingency
import logging

# Suppress all logging messages below CRITICAL
logging.getLogger().setLevel(logging.CRITICAL)

import statsmodels.api as sm 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

def plot_opportunitystate_counts_and_proportions(df, target="OpportunityState"):
    
    counts = df[target].value_counts()
    proportions = counts / counts.sum()

    custom_colors = ["#5B2C6F", "#F5B041"]  # color

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    sns.barplot(
        x=counts.index, 
        y=counts.values, 
        palette=custom_colors, 
        ax=axes[0], 
        edgecolor="black"
    )
    axes[0].set_title("Fordeling af Target-variabel: OpportunityState (antal)")
    axes[0].set_xlabel(target)
    axes[0].set_ylabel("Antal")

    for p in axes[0].patches:
        height = p.get_height()
        axes[0].annotate(
            f"{height}", 
            (p.get_x() + p.get_width()/2, height),
            ha="center", 
            va="bottom", 
            fontsize=11
        )

    sns.barplot(
        x=proportions.index, 
        y=proportions.values, 
        palette=custom_colors, 
        ax=axes[1], 
        edgecolor="black"
    )
    axes[1].set_title("Fordeling af Target-variabel: OpportunityState (andel)")
    axes[1].set_xlabel(target)
    axes[1].set_ylabel("Andel")

    for p in axes[1].patches:
        height = p.get_height()
        axes[1].annotate(
            f"{height*100:.1f}%", 
            (p.get_x() + p.get_width()/2, height),
            ha="center", 
            va="bottom", 
            fontsize=11
        )

    fig.suptitle(
        "Der er en stærk overvægt af vundne salgsmuligheder i datasættet",
        fontsize=16,
        weight="bold"
    )

    
    fig.text(0.02, -0.02, "Kilde: Egen analyse baseret på solutions_ml", fontsize=10)

    plt.tight_layout()
    plt.show()



plot_opportunitystate_counts_and_proportions(solutions_ml)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

def kde_three_features(df, target="OpportunityState"):
    features = ["TotalContractValueDKK", "AnnualContractValueDKK", "MarginValueDKK"]

    fig, axes = plt.subplots(1, 3, figsize=(30 , 10))        #     20, 6 før

    for ax, feature in zip(axes, features):
        sns.kdeplot(
            data=df,
            x=feature,
            hue=target,
            fill=True,
            common_norm=False,
            alpha=0.5,
            linewidth=2,
            ax=ax
        )

        ax.set_title(f"Distribution af {feature}", fontsize=13, weight="bold")
        ax.set_xlabel(feature)
        ax.set_ylabel("Densitet")

        # removing scientific notation
        ax.ticklabel_format(style='plain', useOffset=False, axis='both')

    fig.suptitle("Den samme tendens er gældende i de finansielle tal", fontsize=16, weight="bold")
    fig.text(0.02, -0.02, "Kilde: Egen analyse baseret på solutions_ml", fontsize=10)

    plt.tight_layout()
    plt.show()

kde_three_features(solutions_ml)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

df = solutions_ml.copy()

target = "OpportunityState_num"


numvars = df.select_dtypes(include=[np.number]).columns.tolist()
numvars = [v for v in numvars if v != target]

def kde_numeric_feature(df, feature, target="OpportunityState_num"):
    plt.figure(figsize=(10, 6))

    sns.kdeplot(
        data=df,
        x=feature,
        hue=target,
        fill=True,
        common_norm=False,
        alpha=0.5,
        linewidth=2
    )

    plt.title(f"KDE-distribution af {feature} fordelt på target", fontsize=14, weight="bold")
    plt.xlabel(feature)
    plt.ylabel("Densitet")

    plt.tight_layout()
    plt.show()

for feature in numvars:
    print(f"Plotter KDE for: {feature}")
    kde_numeric_feature(df, feature, target)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"


def cat_counts_and_proportions(df, col, target="OpportunityState", top_n=10):
    contingency = pd.crosstab(df[col].astype(str), df[target])

    # Top N sorting
    contingency["__total__"] = contingency.sum(axis=1)
    contingency = contingency.sort_values("__total__", ascending=False)

    if contingency.shape[0] > top_n:
        other = contingency.iloc[top_n:].sum()
        other.name = "Other"
        contingency = pd.concat([contingency.iloc[:top_n], other.to_frame().T])

    contingency_counts = contingency.drop(columns="__total__")
    contingency_prop = contingency_counts.div(contingency_counts.sum(axis=1), axis=0)

    fig, axes = plt.subplots(
        1, 2,
        figsize=(18, max(6, contingency_counts.shape[0] * 0.5)),
        sharey=True
    )

    palette = sns.color_palette("tab20")

    # COUNTS
    contingency_counts.plot(
        kind="barh",
        stacked=True,
        ax=axes[0],
        color=palette
    )
    axes[0].invert_yaxis()
    axes[0].set_title(f"{col} fordelt på {target} (Counts)", fontsize=14, weight="bold")
    axes[0].set_xlabel("Antal")
    axes[0].set_ylabel(col)

    # PROPORTIONS
    contingency_prop.plot(
        kind="barh",
        stacked=True,
        ax=axes[1],
        color=palette
    )
    axes[1].invert_yaxis()
    axes[1].set_title(f"{col} fordelt på {target} (Proportioner)", fontsize=14, weight="bold")
    axes[1].set_xlabel("Proportion")
    axes[1].set_ylabel("")
    axes[1].legend(title=target, bbox_to_anchor=(1.05, 1), loc="upper left")

    plt.tight_layout()
    plt.show()


categorical_cols = solutions_ml.select_dtypes(include=["object", "category"]).columns.tolist()

target = "OpportunityState"

for col in categorical_cols:
    if col == target:
        continue  # Skip target var
    print(f"Plotter: {col}")
    cat_counts_and_proportions(solutions_ml, col, target)


In [ ]:
# First doing the typical inspection, making sure everything is as it is supposed to post cleaning. 

solutions_ml.head()
solutions_ml.tail()
solutions_ml.info()
solutions_ml.shape
solutions_ml.columns
solutions_ml.dtypes

# checking NA / NaN's 

solutions_ml.isna().sum()
solutions_ml.isna().sum().sum()
solutions_ml.isna().mean()

# Checking for duplicates

solutions_ml.duplicated.any()
solutions_ml.duplicated.sum()
#  potentally extract duplicated rows with solutions_ml[solutions_ml.duplicated()]
solutions_ml.drop_duplicates()

# Summary stats

solutions_ml.describe()
    # in case it misses some columns due to mixed datatypes; 
solutions_ml.describe(include = 'all')
solutions_ml.nunique()
solutions_ml['OpportunityState'].value_counts()




In [ ]:
# Exploring distributions and visualizations of given target + features
    # esp. looking for tendencies that could indicate correlation or atleast a relationship with the target_val
    # Also remember doing for example chi^2 + logreg to give initial indications of statistical significance 

solutions_ml.dtypes

In [ ]:
# Since multicol is only for numvars i use chi^2 for catvars


# CHI^2


target = "OpportunityState"

catvars = [col for col in solutions_ml.select_dtypes(include=["object", "category"]).columns
           if col not in ['OpportunityStatusGroup', 'OpportunityNumber']]


chi2_results = []

for col in catvars:
    contingency = pd.crosstab(solutions_ml[col], solutions_ml[target])
    chi2, p, dof, expected = chi2_contingency(contingency)
    
    chi2_results.append({
        "variable": col,
        "chi2": chi2,
        "p_value": p,
        "degrees_of_freedom": dof
    })

chi2_df = pd.DataFrame(chi2_results).sort_values("p_value")


chi2_df


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

df = solutions_ml.copy()

# Target: skal være 0/1 for logit
y = df["OpportunityState_num"]


numvars = df.select_dtypes(include=[np.number]).columns.tolist()
numvars = [v for v in numvars if v not in ["OpportunityState_num"]] 


def logistic_reg_plot(df, feature, y):
    X = df[[feature]].copy()

   
    X_sm = sm.add_constant(X) 
    sm_model = sm.Logit(y, X_sm).fit(disp=False)

   
    sk_model = LogisticRegression()
    sk_model.fit(X_sm, y)  

    # Range til sigmoid
    x_range = np.linspace(X[feature].min(), X[feature].max(), 300)
    X_pred = pd.DataFrame({
        "const": 1,
        feature: x_range
    })
    y_pred_prob = sm_model.predict(X_pred)

  
    plt.figure(figsize=(10, 6))
    plt.plot(x_range, y_pred_prob, linewidth=3, color="steelblue")
    plt.xlabel(feature)
    plt.ylabel("Probability (Won)")
    plt.title(f"Logistic Regression Sigmoid Curve for {feature}", fontsize=14, weight="bold")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    return sm_model



all_results = {}

for feature in numvars:
    print("\n=====================================")
    print(f"LOGISTIC REGRESSION FOR: {feature}")
    print("=====================================\n")

    model = logistic_reg_plot(df, feature, y)
    all_results[feature] = model

    # Statsmodels summary
    print(model.summary())


In [ ]:
# Corrplot to check correlation between target var + numvars aswell as checking for multicol
# Although decision trees are naturally prone to multicol it never hurts to test and understand the data more nuanced


corr_solutions = solutions_ml.copy()

corr_solutions['OpportunityState'] = corr_solutions['OpportunityState'].map({
    'Won': 1,
    'Lost': 0
})


numvars = [f for f in corr_solutions.columns if f not in catvars
and f not in ['OpportunityStatusGroup', 'OpportunityNumber', 'CloseDate']]
numvars.append('OpportunityState')

corr_df = corr_solutions[numvars]


# Compute correlation matrix (Pearson by default)
corr_matrix = corr_df.corr()

# Plot

fig,ax = plt.subplots(figsize = (10,8))

mask = np.triu(np.ones_like(corr_matrix, dtype = bool), k = 1)  

sns.heatmap(
    corr_matrix,    
    mask = mask,
    annot = False,
    fmt = ".2f",
    cmap = "RdBu_r",
    center = 0,
    vmin = -1,
    vmax = 1,
    square = True,
    linewidth = 0.5,
    annot_kws={"size": 8, "color": "black", "weight": "bold"},
    ax = ax,
    clip_on = False
)

plt.title("Korrelationsmatrice over datasættets numeriske variable", fontsize = 14, fontweight = "bold")
plt.tight_layout()
plt.show()




#### Data vizualisation ####


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

def stacked_counts_and_proportions(df, col, target="OpportunityState", top_n=15):

    contingency = pd.crosstab(df[col].astype(str), df[target])

    contingency["__total__"] = contingency.sum(axis=1)
    contingency = contingency.sort_values("__total__", ascending=False).head(top_n)
    contingency_counts = contingency.drop(columns="__total__")

    contingency_prop = contingency_counts.div(contingency_counts.sum(axis=1), axis=0)

    fig, axes = plt.subplots(
        1, 2,
        figsize=(18, max(6, top_n * 0.5)),
        sharey=True
    )

    palette = sns.color_palette("tab20")

    contingency_counts.plot(
        kind="barh",
        stacked=True,
        ax=axes[0],
        color=palette
    )
    axes[0].invert_yaxis()
    axes[0].set_title(f"Top {top_n} {col} fordelt på {target} (Counts)", fontsize=14, weight="bold")
    axes[0].set_xlabel("Antal")
    axes[0].set_ylabel(col)

    contingency_prop.plot(
        kind="barh",
        stacked=True,
        ax=axes[1],
        color=palette
    )
    axes[1].invert_yaxis()
    axes[1].set_title(f"Top {top_n} {col} fordelt på {target} (Procent)", fontsize=14, weight="bold")
    axes[1].set_xlabel("Procent")
    axes[1].set_ylabel("")

    axes[1].legend(title=target, bbox_to_anchor=(1.05, 1), loc="upper left")

    fig.suptitle(
        f"Distribution af kategorien '{col}' opdelt på target-variablen '{target}'",
        fontsize=16
    )
    fig.text(0.02, -0.02, "Kilde: Egen analyse baseret på solutions_ml", fontsize=10)

    plt.tight_layout()
    plt.show()




stacked_counts_and_proportions(solutions_ml, 'EmployeeName')


In [ ]:
catvars = df.select_dtypes(include=["object", "category"]).columns.tolist()
exclude = ['OpportunityStatusGroup', 'OpportunityNumber', 'OpportunityState']
catvars = [c for c in catvars if c not in exclude]

for col in catvars:
    stacked_counts_and_proportions(df, col, target="OpportunityState", top_n=10)


In [ ]:
## Med udgangspunkt i foregående analyse, fremvises følgende plots som værende af interesse
# ['TotalContractValueDKK', AnnualContractValueDKK, 'MarginValueDKK', 'DaysBetweenCreateClose']
# ['MonthName', 'EmployeeName', 'OpportunityConsciaCountry' (top_n == 7), 'DealSizeName', 'TechnologyName']

In [ ]:
# Target var count / distri 

plot_opportunitystate_counts_and_proportions(solutions_ml)


In [ ]:
# KDE PLOT af de 3 finansielle drenge

kde_three_features(solutions_ml)


In [ ]:
# KDE PLOT af DaysBetweenCreateClose

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

numeric_var = "DaysBetweenCreateClose"
target = "OpportunityState"

plt.figure(figsize=(10, 6))

sns.kdeplot(
    data=solutions_ml,
    x=numeric_var,
    hue=target,
    fill=True,
    common_norm=False,
    alpha=0.5,
    linewidth=2
)

plt.title("Langt de fleste salgsmuligheder lukkes forud for 100 dage. Tabte salgsmulighder er meget mere spredte",
          fontsize=15, weight="bold")
plt.xlabel("DaysBetweenCreateClose")
plt.ylabel("Densitet")

plt.figtext(0.01, -0.02, "Kilde: Egen analyse baseret på solutions_ml", fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
# Month

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

def monthname_counts_and_proportions(df, col="MonthName", target="OpportunityState"):

    month_order = [
        "January", "February", "March", "April", "May", "June",
        "July", "August", "September", "October", "November", "December"
    ]

    df[col] = pd.Categorical(df[col], categories=month_order, ordered=True)

    contingency = pd.crosstab(df[col], df[target])

    contingency = contingency.loc[month_order].dropna(how="all")

    contingency_prop = contingency.div(contingency.sum(axis=1), axis=0)

    fig, axes = plt.subplots(
        1, 2,
        figsize=(18, 8),
        sharey=True
    )

    palette = sns.color_palette("tab20")

    # ---- COUNTS ----
    contingency.plot(
        kind="barh",
        stacked=True,
        ax=axes[0],
        color=palette
    )
    axes[0].invert_yaxis()
    axes[0].set_title(f"{col} fordelt på {target} (Antal)", fontsize=14, weight="bold")
    axes[0].set_xlabel("Antal")
    axes[0].set_ylabel(col)

    contingency_prop.plot(
        kind="barh",
        stacked=True,
        ax=axes[1],
        color=palette
    )
    axes[1].invert_yaxis()
    axes[1].set_title(f"{col} fordelt på {target} (Andel)", fontsize=14, weight="bold")
    axes[1].set_xlabel("Andel")
    axes[1].set_ylabel("")

    axes[1].legend(title=target, bbox_to_anchor=(1.05, 1), loc="upper left")

    fig.suptitle(
        f"Der er usædvanligt store sæsonudsving i antal lukninger af salgsmuligheder",
        fontsize=16,
        weight="bold"
    )
    fig.text(0.02, -0.02, "Kilde: Egen analyse baseret på solutions_ml", fontsize=10)

    plt.tight_layout()
    plt.show()


monthname_counts_and_proportions(solutions_ml, col="MonthName", target="OpportunityState")


In [ ]:
Employee_IDs = spark.read.format("delta").load("cantshow")


In [ ]:
solutions_ml = solutions_ml.toPandas()
Employee_IDs = Employee_IDs.toPandas()

Employee_IDs_unique = Employee_IDs.drop_duplicates(subset=["EmployeeName"])

name_to_id = Employee_IDs_unique.set_index("EmployeeName")["EmployeeID"].to_dict()

solutions_ml["EmployeeID"] = solutions_ml["EmployeeName"].map(name_to_id)

unmatched = solutions_ml[solutions_ml["EmployeeID"].isna()]["EmployeeName"].unique()
print("Unmatched names:", unmatched)


In [ ]:
# Barplot af EmployeeName

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

def employee_counts_and_proportions(df, col="EmployeeID", target="OpportunityState", top_n=10):

    contingency = pd.crosstab(df[col].astype(str), df[target])

    contingency["__total__"] = contingency.sum(axis=1)
    contingency = contingency.sort_values("__total__", ascending=False).head(top_n)
    contingency_counts = contingency.drop(columns="__total__")

    contingency_prop = contingency_counts.div(contingency_counts.sum(axis=1), axis=0)

    fig, axes = plt.subplots(
        1, 2,
        figsize=(18, max(6, top_n * 0.5)),
        sharey=True
    )

    palette = sns.color_palette("tab20")

    contingency_counts.plot(
        kind="barh",
        stacked=True,
        ax=axes[0],
        color=palette
    )
    axes[0].invert_yaxis()
    axes[0].set_title(f"{col} fordelt på {target} (Antal)", fontsize=14, weight="bold")
    axes[0].set_xlabel("Antal")
    axes[0].set_ylabel(col)

    contingency_prop.plot(
        kind="barh",
        stacked=True,
        ax=axes[1],
        color=palette
    )
    axes[1].invert_yaxis()
    axes[1].set_title(f"{col} fordelt på {target} (Andel)", fontsize=14, weight="bold")
    axes[1].set_xlabel("Andel")
    axes[1].set_ylabel("")


    fig.suptitle(
        f"Der er stor forskel på medarbejdernes andel af vundne salgsmuligheder",
        fontsize=16,
        weight="bold"
    )
    fig.text(0.02, -0.02, "Kilde: Egen analyse baseret på solutions_ml", fontsize=10)

    plt.tight_layout()
    plt.show()


employee_counts_and_proportions(solutions_ml, col="EmployeeID", target="OpportunityState", top_n=10)


In [ ]:
# Barplot af OpportunityConsciaCountry (top_n == 7)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

def country_counts_and_proportions(df, col="OpportunityConsciaCountry", target="OpportunityState", top_n=7):

    contingency = pd.crosstab(df[col].astype(str), df[target])

    contingency["__total__"] = contingency.sum(axis=1)
    contingency = contingency.sort_values("__total__", ascending=False).head(top_n)
    contingency_counts = contingency.drop(columns="__total__")

    contingency_prop = contingency_counts.div(contingency_counts.sum(axis=1), axis=0)

    fig, axes = plt.subplots(
        1, 2,
        figsize=(22, max(6, top_n * 0.5)),
        sharey=True
    )

    palette = sns.color_palette("tab20")

    contingency_counts.plot(
        kind="barh",
        stacked=True,
        ax=axes[0],
        color=palette
    )
    axes[0].invert_yaxis()
    axes[0].set_title(f"{col} fordelt på {target} (Antal)", fontsize=14, weight="bold")
    axes[0].set_xlabel("Antal")
    axes[0].set_ylabel(col)

    contingency_prop.plot(
        kind="barh",
        stacked=True,
        ax=axes[1],
        color=palette
    )
    axes[1].invert_yaxis()
    axes[1].set_title(f"{col} fordelt på {target} (Andel)", fontsize=14, weight="bold")
    axes[1].set_xlabel("Andel")
    axes[1].set_ylabel("")

    axes[1].legend(title=target, bbox_to_anchor=(1.05, 1), loc="upper left")

    fig.suptitle(
        f"Der opleves en stor variation i både antal såvel som andel af lukkede salgsmuligheder fordelt på land",
        fontsize=16,
        weight="bold"
    )
    fig.text(0.02, -0.02, "Kilde: Egen analyse baseret på solutions_ml", fontsize=10)

    plt.tight_layout()
    plt.show()


country_counts_and_proportions(solutions_ml, col="OpportunityConsciaCountry", target="OpportunityState", top_n=7)


In [ ]:
# Barplot af DealSizeName

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

def dealsize_counts_and_proportions(df, col="DealSizeName", target="OpportunityState"):

    contingency = pd.crosstab(df[col].astype(str), df[target])

    contingency["__total__"] = contingency.sum(axis=1)
    contingency = contingency.sort_values("__total__", ascending=False)
    contingency_counts = contingency.drop(columns="__total__")

    contingency_prop = contingency_counts.div(contingency_counts.sum(axis=1), axis=0)

    fig, axes = plt.subplots(
        1, 2,
        figsize=(18, max(6, len(contingency) * 0.5)),
        sharey=True
    )

    palette = sns.color_palette("tab20")

    contingency_counts.plot(
        kind="barh",
        stacked=True,
        ax=axes[0],
        color=palette
    )
    axes[0].invert_yaxis()
    axes[0].set_title(f"{col} fordelt på {target} (Antal)", fontsize=14, weight="bold")
    axes[0].set_xlabel("Andel")
    axes[0].set_ylabel(col)

    contingency_prop.plot(
        kind="barh",
        stacked=True,
        ax=axes[1],
        color=palette
    )
    axes[1].invert_yaxis()
    axes[1].set_title(f"{col} fordelt på {target} (Andel)", fontsize=14, weight="bold")
    axes[1].set_xlabel("Andel")
    axes[1].set_ylabel("")

    axes[1].legend(title=target, bbox_to_anchor=(1.05, 1), loc="upper left")

    fig.suptitle(
        f"Deals < 50.000 kr. fremstår mest frekvente og med den stærkeste relative succesrate.",
        fontsize=16,
        weight="bold"
    )
    fig.text(0.02, -0.02, "Kilde: Egen analyse baseret på solutions_ml", fontsize=10)

    plt.tight_layout()
    plt.show()


dealsize_counts_and_proportions(solutions_ml, col="DealSizeName", target="OpportunityState")


In [ ]:
# Barplot af TechnologyName

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

def technology_counts_and_proportions(df, col="TechnologyName", target="OpportunityState", top_n=5):

    contingency = pd.crosstab(df[col].astype(str), df[target])

    contingency["__total__"] = contingency.sum(axis=1)
    contingency = contingency.sort_values("__total__", ascending=False).head(top_n)
    contingency_counts = contingency.drop(columns="__total__")

    contingency_prop = contingency_counts.div(contingency_counts.sum(axis=1), axis=0)

    fig, axes = plt.subplots(
        1, 2,
        figsize=(18, max(6, top_n * 0.6)),
        sharey=True
    )

    palette = sns.color_palette("tab20")

    contingency_counts.plot(
        kind="barh",
        stacked=True,
        ax=axes[0],
        color=palette
    )
    axes[0].invert_yaxis()
    axes[0].set_title(f"{col} fordelt på {target} (Antal)", fontsize=14, weight="bold")
    axes[0].set_xlabel("Antal")
    axes[0].set_ylabel(col)

    contingency_prop.plot(
        kind="barh",
        stacked=True,
        ax=axes[1],
        color=palette
    )
    axes[1].invert_yaxis()
    axes[1].set_title(f"{col} fordelt på {target} (Andel)", fontsize=14, weight="bold")
    axes[1].set_xlabel("Andel")
    axes[1].set_ylabel("")

    axes[1].legend(title=target, bbox_to_anchor=(1.05, 1), loc="upper left")

    fig.suptitle(
        f"Network er det klart mest frekvente forretningsområde, og har den stærkste relative succesrate ",
        fontsize=16,
        weight="bold"
    )
    fig.text(0.02, -0.02, "Kilde: Egen analyse baseret på solutions_ml", fontsize=10)

    plt.tight_layout()
    plt.show()


technology_counts_and_proportions(solutions_ml, col="TechnologyName", target="OpportunityState", top_n=5)




In [ ]:
# KDE PLOT af DaysBetweenCreateClose

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

numeric_var = "TCV_Margin_Ratio"
target = "OpportunityState"

plt.figure(figsize=(10, 6))

sns.kdeplot(
    data=solutions_ml,
    x=numeric_var,
    hue=target,
    fill=True,
    common_norm=False,
    alpha=0.5,
    linewidth=2
)

plt.title("Langt de fleste salgsmuligheder lukkes forud for 100 dage. Tabte salgsmulighder er meget mere spredte",
          fontsize=15, weight="bold")
plt.xlabel("DaysBetweenCreateClose")
plt.ylabel("Densitet")

plt.figtext(0.01, -0.02, "Kilde: Egen analyse baseret på solutions_ml", fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
Quarter
EOQ
SubType 
TCV_Margin_Ratio

In [ ]:
# Barplot af SubType

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

def technology_counts_and_proportions(df, col="SubType", target="OpportunityState", top_n=5):

    contingency = pd.crosstab(df[col].astype(str), df[target])

    contingency["__total__"] = contingency.sum(axis=1)
    contingency = contingency.sort_values("__total__", ascending=False).head(top_n)
    contingency_counts = contingency.drop(columns="__total__")

    contingency_prop = contingency_counts.div(contingency_counts.sum(axis=1), axis=0)

    fig, axes = plt.subplots(
        1, 2,
        figsize=(18, max(6, top_n * 0.6)),
        sharey=True
    )

    palette = sns.color_palette("tab20")

    contingency_counts.plot(
        kind="barh",
        stacked=True,
        ax=axes[0],
        color=palette
    )
    axes[0].invert_yaxis()
    axes[0].set_title(f"{col} fordelt på {target} (Antal)", fontsize=14, weight="bold")
    axes[0].set_xlabel("Antal")
    axes[0].set_ylabel(col)

    contingency_prop.plot(
        kind="barh",
        stacked=True,
        ax=axes[1],
        color=palette
    )
    axes[1].invert_yaxis()
    axes[1].set_title(f"{col} fordelt på {target} (Andel)", fontsize=14, weight="bold")
    axes[1].set_xlabel("Andel")
    axes[1].set_ylabel("")

    axes[1].legend(title=target, bbox_to_anchor=(1.05, 1), loc="upper left")

    fig.suptitle(
        f"Der opleves større udsving i den relative succesrate for de forskellige forretningsundertyper ",
        fontsize=16,
        weight="bold"
    )
    fig.text(0.02, -0.02, "Kilde: Egen analyse baseret på solutions_ml", fontsize=10)

    plt.tight_layout()
    plt.show()


technology_counts_and_proportions(solutions_ml, col="SubType", target="OpportunityState", top_n=5)




In [ ]:
# Barplot af Quarter

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

def technology_counts_and_proportions(df, col="Quarter", target="OpportunityState", top_n=5):

    contingency = pd.crosstab(df[col].astype(str), df[target])

    contingency["__total__"] = contingency.sum(axis=1)
    contingency = contingency.sort_values("__total__", ascending=False).head(top_n)
    contingency_counts = contingency.drop(columns="__total__")

    contingency_prop = contingency_counts.div(contingency_counts.sum(axis=1), axis=0)

    fig, axes = plt.subplots(
        1, 2,
        figsize=(18, max(6, top_n * 0.6)),
        sharey=True
    )

    palette = sns.color_palette("tab20")

    contingency_counts.plot(
        kind="barh",
        stacked=True,
        ax=axes[0],
        color=palette
    )
    axes[0].invert_yaxis()
    axes[0].set_title(f"{col} fordelt på {target} (Antal)", fontsize=14, weight="bold")
    axes[0].set_xlabel("Antal")
    axes[0].set_ylabel(col)

    contingency_prop.plot(
        kind="barh",
        stacked=True,
        ax=axes[1],
        color=palette
    )
    axes[1].invert_yaxis()
    axes[1].set_title(f"{col} fordelt på {target} (Andel)", fontsize=14, weight="bold")
    axes[1].set_xlabel("Andel")
    axes[1].set_ylabel("")

    axes[1].legend(title=target, bbox_to_anchor=(1.05, 1), loc="upper left")

    fig.suptitle(
        f"Omkring halvdelen af alle salg bliver lukket i det første kvartal. ",
        fontsize=16,
        weight="bold"
    )
    fig.text(0.02, -0.02, "Kilde: Egen analyse baseret på solutions_ml", fontsize=10)

    plt.tight_layout()
    plt.show()


technology_counts_and_proportions(solutions_ml, col="Quarter", target="OpportunityState", top_n=5)




In [ ]:
# Barplot af Is_EOQ

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

def technology_counts_and_proportions(df, col="Is_EOQ", target="OpportunityState", top_n=5):

    contingency = pd.crosstab(df[col].astype(str), df[target])

    contingency["__total__"] = contingency.sum(axis=1)
    contingency = contingency.sort_values("__total__", ascending=False).head(top_n)
    contingency_counts = contingency.drop(columns="__total__")

    contingency_prop = contingency_counts.div(contingency_counts.sum(axis=1), axis=0)

    fig, axes = plt.subplots(
        1, 2,
        figsize=(18, max(6, top_n * 0.6)),
        sharey=True
    )

    palette = sns.color_palette("tab20")

    contingency_counts.plot(
        kind="barh",
        stacked=True,
        ax=axes[0],
        color=palette
    )
    axes[0].invert_yaxis()
    axes[0].set_title(f"{col} fordelt på {target} (Antal)", fontsize=14, weight="bold")
    axes[0].set_xlabel("Antal")
    axes[0].set_ylabel(col)

    contingency_prop.plot(
        kind="barh",
        stacked=True,
        ax=axes[1],
        color=palette
    )
    axes[1].invert_yaxis()
    axes[1].set_title(f"{col} fordelt på {target} (Andel)", fontsize=14, weight="bold")
    axes[1].set_xlabel("Andel")
    axes[1].set_ylabel("")

    axes[1].legend(title=target, bbox_to_anchor=(1.05, 1), loc="upper left")

    fig.suptitle(
        f"Flest salg bliver lukket i slutningen af kvartaler, trods en meget ens relativ succesrate.",
        fontsize=16,
        weight="bold"
    )
    fig.text(0.02, -0.02, "Kilde: Egen analyse baseret på solutions_ml", fontsize=10)

    plt.tight_layout()
    plt.show()


technology_counts_and_proportions(solutions_ml, col="Is_EOQ", target="OpportunityState", top_n=5)


